In [7]:
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
import re

In [8]:
with open("../data/nubase_4.mas20.txt", "r") as f:
    lines = f.readlines()


half_life_units_factors: dict[str, float] = {
    "ys": 1e-24,
    "zs": 1e-21,
    "as": 1e-18,
    "fs": 1e-15,
    "ps": 1e-12,
    "ns": 1e-9,
    "us": 1e-6,
    "ms": 1e-3,
    "s": 1,
    "m": 60,
    "h": 60 * 60,
    "d": 24 * 60 * 60,
    "y": 31557600,
    "ky": 31557600 * 1e3,
    "My": 31557600 * 1e6,
    "Gy": 31557600 * 1e9,
    "Ty": 31557600 * 1e12,
    "Py": 31557600 * 1e15,
    "Ey": 31557600 * 1e18,
    "Yy": 31557600 * 1e21,
    "Zy": 31557600 * 1e24,
}


def convert_half_life_to_s(half_life: float, unit: str) -> float:
    if unit not in half_life_units_factors:
        raise ValueError(f"Unit {unit} not found in half_life_units_factors")
    return half_life * half_life_units_factors[unit]




In [13]:
@dataclass
class Isotope:
    mass_number_AAA: int
    atomic_number_ZZZ: int
    atomic_number_i: int
    nucleide: str
    element: str
    s: str
    mass_excess: float | None  # Mass Excess in keV
    mass_excess_systematics: bool
    mass_excess_uncertainty: float | None  # Mass Excess uncertainty in keV
    mass_excess_uncertainty_systematics: bool
    # isomer_excitation_energy: float | None = None  # Isomer excitation energy in keV
    # isomer_excitation_energy_systematics: bool
    # isomer_excitation_energy_uncertainty: float | None = (
    #     None  # Isomer excitation energy uncertainty in keV
    # )
    # isomer_excitation_energy_uncertainty_systematics: bool
    # origin_of_excitation_energy: str = ""

    stable: bool
    p_unstable: bool
    half_life: float | None  # Half-life in s
    # half_life_systematics: bool
    # half_life_uncertainty: float | None  # Half-life uncertainty in s
    half_life_str: str

    nubase2020_line: str = ""

    isomers: list["Isotope"] = field(default_factory=list)
    isobaric_analog_states: list["Isotope"] = field(default_factory=list)

    def for_json(self):
        return {
            "mass_number_AAA": self.mass_number_AAA,
            "atomic_number_ZZZ": self.atomic_number_ZZZ,
            "atomic_number_i": self.atomic_number_i,
            "nucleide": self.nucleide,
            "element": self.element,
            "s": self.s,
            "mass_excess": self.mass_excess,
            "mass_excess_systematics": self.mass_excess_systematics,
            "mass_excess_uncertainty": self.mass_excess_uncertainty,
            "mass_excess_uncertainty_systematics": self.mass_excess_uncertainty_systematics,
            "stable": self.stable,
            "p_unstable": self.p_unstable,
            "half_life": self.half_life,
            "half_life_str": self.half_life_str,
            "nubase2020_line": self.nubase2020_line,
            "isomers": [i.for_json() for i in self.isomers],
            "isobaric_analog_states": [i.for_json() for i in self.isobaric_analog_states],
        }


def isotope_from_line(line):
    mass_number_AAA = int(line[0:3])
    atomic_number_ZZZ = int(line[4:7])
    atomic_number_i = int(line[7:8])
    element_A = line[11:16].strip()
    s = line[16:17].strip()

    mass_excess = line[18:31].strip()
    mass_excess_r = mass_excess.replace("#", "")
    mass_excess_systematics = mass_excess != mass_excess_r
    if not mass_excess_r == "":
        mass_excess = float(mass_excess_r)

    mass_excess_uncertainty = line[31:42].strip()
    mass_excess_uncertainty_r = mass_excess_uncertainty.replace("#", "")
    mass_excess_uncertainty_systematics = (
        mass_excess_uncertainty == mass_excess_uncertainty_r
    )
    if not mass_excess_uncertainty_r == "":
        mass_excess_uncertainty = float(mass_excess_uncertainty_r)

    half_life_raw = line[69:78].strip()
    half_life_unit = line[78:80].strip()

    stable = False
    p_unstable = False

    if half_life_raw == "stbl":
        half_life = None
        stable = True
    elif half_life_raw == "p-unst":
        half_life = None
        p_unstable = True
    elif half_life_raw == "":
        half_life = None
    else:
        hlnstr = re.sub(r"[^0-9.]", "", half_life_raw)
        half_life = convert_half_life_to_s(float(hlnstr), half_life_unit)

    i = Isotope(
        nubase2020_line=line,
        mass_number_AAA=mass_number_AAA,
        atomic_number_ZZZ=atomic_number_ZZZ,
        atomic_number_i=atomic_number_i,
        nucleide=element_A,
        element=re.sub(r"[0-9]", "", element_A),
        s=s,
        mass_excess=mass_excess,
        mass_excess_systematics=mass_excess_systematics,
        mass_excess_uncertainty=mass_excess_uncertainty,
        mass_excess_uncertainty_systematics=mass_excess_uncertainty_systematics,
        half_life_str=f"{half_life_raw} {half_life_unit}".strip(),
        half_life=half_life,
        stable=stable,
        p_unstable=p_unstable,
    )
    return i


isotopes: list[Isotope] = []
isotope_lookup: dict[str, Isotope] = {}

for line in lines:
    if line[0] == "#":
        continue
    i = isotope_from_line(line)

    if i.s in ["i", "j"]:
        isotope_lookup[i.nucleide].isobaric_analog_states.append(i)
    elif i.s in ["m", "n"]:
        isotope_lookup[i.nucleide].isomers.append(i)
    else:
        isotopes.append(i)
        isotope_lookup[i.nucleide] = i

# isotopes

# convert isotopes to json
import json

isotopes_json = [i.for_json() for i in isotopes]
with open("../docs/data/nubase_4.mas20.json", "w") as f:
    json.dump(isotopes_json, f, indent=2)


isotope_lookup

{'1n': Isotope(mass_number_AAA=1, atomic_number_ZZZ=0, atomic_number_i=0, nucleide='1n', element='n', s='', mass_excess=8071.3181, mass_excess_systematics=False, mass_excess_uncertainty=0.0004, mass_excess_uncertainty_systematics=True, stable=False, p_unstable=False, half_life=609.8, half_life_str='609.8 s', nubase2020_line='001 0000   1n       8071.3181     0.0004                              609.8    s 0.6    1/2+*         06          1932 B-=100\n', isomers=[], isobaric_analog_states=[]),
 '1H': Isotope(mass_number_AAA=1, atomic_number_ZZZ=1, atomic_number_i=0, nucleide='1H', element='H', s='', mass_excess=7288.971064, mass_excess_systematics=False, mass_excess_uncertainty=1.3e-05, mass_excess_uncertainty_systematics=True, stable=True, p_unstable=False, half_life=None, half_life_str='stbl', nubase2020_line='001 0010   1H       7288.971064   0.000013                            stbl              1/2+*         06          1920 IS=99.9855 78\n', isomers=[], isobaric_analog_states=[]),
 